In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

device = torch.device('cuda')

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation").to(device)
nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/411k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/532M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
content=" giao đồ rất đẹp tỉ mĩ nên cẩn thận và tốt nhưng hơi xấu một chút"

ner_results = nlp(content)
example_tok = ""
for e in ner_results:
    if "##" in e["word"]:
        example_tok = example_tok + e["word"].replace("##","")
    elif e["entity"] =="I":
        example_tok = example_tok + "_" + e["word"]
    elif ("." in e["word"]) or ("/" in e["word"]) or ("%" in e["word"]) or ("-" in e["word"]):
        example_tok = example_tok + "" + e["word"]
    else:
        example_tok = example_tok + " " + e["word"]

print(example_tok)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 giao đồ rất đẹp tỉ_mĩ nên cẩn_thận và tốt nhưng hơi xấu một_chút


In [7]:
# Hàm tiền xử lý cho text phù hợp với dataset
def word_weg(text, nlp):
  ner_results = nlp(text)
  example_tok = ""
  for e in ner_results:
      if "##" in e["word"]:
          example_tok = example_tok + e["word"].replace("##","")
      elif e["entity"] =="I":
          example_tok = example_tok + "_" + e["word"]
      elif ("." in e["word"]) or ("/" in e["word"]) or ("%" in e["word"]) or ("-" in e["word"]):
        example_tok = example_tok + "" + e["word"]
      else:
        example_tok = example_tok + " " + e["word"]
  return example_tok

def split_sentence_into_chunks(sentence, max_length=512):
    """Cắt câu dài thành các đoạn nhỏ hơn hoặc bằng max_length ký tự."""
    chunks = []
    while len(sentence) > max_length:
        # Tìm vị trí dấu cách gần nhất trước giới hạn max_length
        split_pos = sentence[:max_length].rfind(" ")
        if split_pos == -1:  # Nếu không tìm thấy dấu cách
            split_pos = max_length
        chunks.append(sentence[:split_pos].strip())
        sentence = sentence[split_pos:].strip()
    if sentence:  # Thêm phần còn lại của câu
        chunks.append(sentence)
    return chunks

def split_text_with_long_sentences(text, max_length=512):
    """Cắt văn bản thành nhiều đoạn nhỏ hơn hoặc bằng max_length ký tự, bao gồm cả câu dài."""
    text = text.strip()
    sentences = re.split(r'(?<=[.])', text)  # Tách theo câu, giữ lại dấu chấm
    chunks = []
    for sentence in sentences:
        sentence = sentence.strip()
        if len(sentence) > max_length:
            # Nếu câu vượt quá max_length, cắt thành các đoạn nhỏ
            chunks.extend(split_sentence_into_chunks(sentence, max_length))
        else:
            # Nếu câu không vượt quá max_length, thêm vào danh sách
            chunks.append(sentence)
    return chunks

# Hàm tiền xử lý cho text
def preprocessing_text(text, nlp):
    # lower text
    text = text.lower()

    # remove special char
    allowed_special_chars = r"%"
    allowed_chars = r"[^a-zA-Z0-9\s" + re.escape(allowed_special_chars) + r"àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ]"
    text = re.sub(allowed_chars, '', text)

    # word segmentor
    text = word_weg(text, nlp)

    # remove stop word
    with open('/content/drive/MyDrive/XLDL/data_v4/vietnamese.txt', 'r', encoding='utf-8') as f:
        stop_words = set(f.read().strip().split('\n'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text =' '.join(filtered_words)

    return text

def preprocessing_data(text, nlp):

    sentences = split_text_with_long_sentences(text, max_length=512)

    # Loại bỏ các câu rỗng hoặc chỉ chứa khoảng trắng
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Nối kết quả sau xử lý bằng dấu cách
    text = " ".join(preprocessing_text(sentence, nlp) for sentence in sentences)
    return text

In [8]:
import pandas as pd

# Đường dẫn tới tệp dữ liệu
input_file_path = "/content/drive/MyDrive/XLDL/data_v4/dataset_v3.csv"



# Đọc dữ liệu từ file CSV
df = pd.read_csv(input_file_path)
df['comment'] = df['comment'].astype(str)

# Kiểm tra xem các cột cần thiết có tồn tại không
required_columns = ['comment', 'label']
if not all(column in df.columns for column in required_columns):
    raise ValueError(f"Tệp CSV phải chứa các cột: {', '.join(required_columns)}")

# In ra tiến độ xử lý
for idx, row in df.iterrows():
    print(f"\nComment thứ {idx}: ",row['comment'])
    print(f"==> Xử lý: ",preprocessing_data(row['comment'], nlp))
    if idx ==100:
        break



Comment thứ 0:  Tôi đã đặt mua sản phẩm này. Mất đến 5 ngày để nhận sản phẩm.Lúc kiểm tra,  hộp sản phẩm móp méo và sản phẩm bị trầy bề mặt đáy bàn ủi. Ngay hôm đó, tôi đã gửi 2 yêu cầu đổi trả trên website và không có phản hồi. Sau đó nhiều lần gọi Hotline, mới gặp được nhân viên hỗ trợ và được hỗ trợ đổi sản phẩm khác. Tôi đã theo sát  và thấy sản phẩm đóng gói và sẵn sàng giao hàng từ ngày 17 /10. Vậy mà tới hôm nay tôi vẫn chưa nhận được sản phẩm. Và càng thất vọng hơn khi đã gửi yêu cầu hỏi tại sao lại chưa giao. Vẫn không có phản hồi. Gọi Hotline thì không ai nghe máy, THẬT SỰ THẤT VỌNG VỀ TIKI.
==> Xử lý:  tôi đã đặt mua sản_phẩm mất đến 5 ngày để nhận sản_phẩm lúc kiểm_tra hộp sản_phẩm móp_méo sản_phẩm bị trầy bề_mặt đáy bàn_ủi ngay hôm tôi đã gửi 2 yêu_cầu đổi_trả trên website không có phản_hồi sau nhiều lần gọi hotline mới gặp được nhân_viên hỗ_trợ được hỗ_trợ đổi sản_phẩm khác tôi đã theo sát thấy sản_phẩm đóng_gói sẵn_sàng giao hàng từ ngày 17 10 vậy_mà tới hôm_nay tôi vẫn

In [ ]:
import pandas as pd

# Đường dẫn tới tệp dữ liệu
input_file_path1 = "/content/drive/MyDrive/XLDL/data_v3/positive.csv"
output_file_path1 = "/content/drive/MyDrive/XLDL/data_v3/positive_process.csv"

# Đường dẫn tới tệp dữ liệu
input_file_path2 = "/content/drive/MyDrive/XLDL/data_v3/negative.csv"
output_file_path2 = "/content/drive/MyDrive/XLDL/data_v3/negative_process.csv"

# Đọc dữ liệu từ file CSV
pos_df = pd.read_csv(input_file_path1)
pos_df['comment'] = pos_df['comment'].astype(str)

# Đọc dữ liệu từ file CSV
neg_df = pd.read_csv(input_file_path2)
neg_df['comment'] = neg_df['comment'].astype(str)

# In ra tiến độ xử lý
for idx, row in pos_df.iterrows():

    pos_df.at[idx, 'comment'] = preprocessing_data(row['comment'], nlp)

    if idx % 100 == 0:  # In ra sau mỗi 100 bản ghi
        print(f"Đang xử lý comment tích cực: {idx}/{len(pos_df)}")

# Lưu lại kết quả vào tệp mới
pos_df.to_csv(output_file_path1, index=False, encoding='utf-8')

print(f"Dữ liệu comment tích cực đã được xử lý và lưu tại: {output_file_path1}")

# In ra tiến độ xử lý
for idx, row in neg_df.iterrows():

    neg_df.at[idx, 'comment'] = preprocessing_data(row['comment'], nlp)

    if idx % 100 == 0:  # In ra sau mỗi 100 bản ghi
        print(f"Đang xử lý comment tiêu cực: {idx}/{len(neg_df)}")

# Lưu lại kết quả vào tệp mới
neg_df.to_csv(output_file_path2, index=False, encoding='utf-8')

print(f"Dữ liệu comment tiêu cực đã được xử lý và lưu tại: {output_file_path2}")


Đang xử lý comment tích cực: 0/14997
Đang xử lý comment tích cực: 100/14997
Đang xử lý comment tích cực: 200/14997
Đang xử lý comment tích cực: 300/14997
Đang xử lý comment tích cực: 400/14997
Đang xử lý comment tích cực: 500/14997
Đang xử lý comment tích cực: 600/14997
Đang xử lý comment tích cực: 700/14997
Đang xử lý comment tích cực: 800/14997
Đang xử lý comment tích cực: 900/14997
Đang xử lý comment tích cực: 1000/14997
Đang xử lý comment tích cực: 1100/14997
Đang xử lý comment tích cực: 1200/14997
Đang xử lý comment tích cực: 1300/14997
Đang xử lý comment tích cực: 1400/14997
Đang xử lý comment tích cực: 1500/14997
Đang xử lý comment tích cực: 1600/14997
Đang xử lý comment tích cực: 1700/14997
Đang xử lý comment tích cực: 1800/14997
Đang xử lý comment tích cực: 1900/14997
Đang xử lý comment tích cực: 2000/14997
Đang xử lý comment tích cực: 2100/14997
Đang xử lý comment tích cực: 2200/14997
Đang xử lý comment tích cực: 2300/14997
Đang xử lý comment tích cực: 2400/14997
Đang xử lý c

In [ ]:

print(pos_df.head(30))

print(neg_df.head(30))

                                              comment  label
0                                      rất ưng còn rẻ      0
1   đóng_gói cha chắc_chắn sp rất đáng tiền giao h...      0
2                 hàng rất tốt giao hàng cũng lịch_sự      0
3          giao hàng nhanh chuột đẹp hơi to tay 1 xíu      0
4   chất_lượng sản_phẩm tuyệt_vời đẹp nhưng mặt lớ...      0
5                        hàng chuẩn đẹp lắm cám_ơn sh      0
6                           đúng danh_bất_hư_truyền        0
7                        nhìn hoàn_thiện có_vẻ đc nhỉ      0
8                         giá tốt sản_phẩm chính hãng      0
9   chất_lượng tuyệt_vời bao_bì đẹp chủ yêu_yêu gó...      0
10  chất_lượng sản_phẩm tuyệt_vời co tang_qua nua ...      0
11  chất_lượng sản_phẩm tuyệt_vời đóng_gói sản_phẩ...      0
12  thời_gian giao hàng rất nhanh giá rẻ giày cực ...      0
13  giao hàng nhanh hàng gần mới lắp phát ăn luân ...      0
14                  phao chất đẹp mềm nói_chung ưng ạ      0
15                      

In [9]:
import pandas as pd

# Đường dẫn tới tệp dữ liệu
input_file_path1 = "/content/drive/MyDrive/XLDL/data_v4/dataset_v3.csv"
output_file_path1 = "/content/drive/MyDrive/XLDL/data_v4/dataset_v4_process.csv"

# Đọc dữ liệu từ file CSV
data_df = pd.read_csv(input_file_path1)
data_df['comment'] = data_df['comment'].astype(str)


# In ra tiến độ xử lý
for idx, row in data_df.iterrows():

    data_df.at[idx, 'comment'] = preprocessing_data(row['comment'], nlp)

    if idx % 100 == 0:  # In ra sau mỗi 100 bản ghi
        print(f"Đang xử lý comment tích cực: {idx}/{len(data_df)}")

# Lưu lại kết quả vào tệp mới
data_df.to_csv(output_file_path1, index=False, encoding='utf-8')

print(f"Dữ liệu comment đã được xử lý và lưu tại: {output_file_path1}")



Đang xử lý comment tích cực: 0/29278
Đang xử lý comment tích cực: 100/29278
Đang xử lý comment tích cực: 200/29278
Đang xử lý comment tích cực: 300/29278
Đang xử lý comment tích cực: 400/29278
Đang xử lý comment tích cực: 500/29278
Đang xử lý comment tích cực: 600/29278
Đang xử lý comment tích cực: 700/29278
Đang xử lý comment tích cực: 800/29278
Đang xử lý comment tích cực: 900/29278
Đang xử lý comment tích cực: 1000/29278
Đang xử lý comment tích cực: 1100/29278
Đang xử lý comment tích cực: 1200/29278
Đang xử lý comment tích cực: 1300/29278
Đang xử lý comment tích cực: 1400/29278
Đang xử lý comment tích cực: 1500/29278
Đang xử lý comment tích cực: 1600/29278
Đang xử lý comment tích cực: 1700/29278
Đang xử lý comment tích cực: 1800/29278
Đang xử lý comment tích cực: 1900/29278
Đang xử lý comment tích cực: 2000/29278
Đang xử lý comment tích cực: 2100/29278
Đang xử lý comment tích cực: 2200/29278
Đang xử lý comment tích cực: 2300/29278
Đang xử lý comment tích cực: 2400/29278
Đang xử lý c